# check torch

In [18]:
import torch

print("CUDA available?", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available? True
Device count: 1
Device name: NVIDIA GeForce RTX 4070 SUPER


# Prepare Dataset

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import pandas as pd
import os
df = pd.read_pickle('data/trainingData.pkl')
miete_col = 'wohn_leer'

df = df.drop(columns=[miete_col])
X = df.drop(columns=['preis_miet_best'], axis=1).values
y = df['preis_miet_best'].values


In [20]:

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)


#  Create Dataset and DataLoader

In [21]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)


# Define the Model Class

In [22]:
import torch.nn as nn


class TabularNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_rate, activation_fn):
        super(TabularNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            if activation_fn == 'relu':
                layers.append(nn.ReLU())
            elif activation_fn == 'tanh':
                layers.append(nn.Tanh())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))  # output 1 value for regression
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze(1)  # output shape: (batch_size,)



#  Define Training and Evaluation Functions

In [23]:
def train_one_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, device):
    model.eval()
    criterion = torch.nn.MSELoss()
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            batch_size = y_batch.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

    avg_loss = total_loss / total_samples
    return avg_loss


# Define the Optuna Objective Function

In [24]:
import optuna

def objective(trial):
    # Sample hyperparameters
    hidden_dim1 = trial.suggest_categorical("hidden_dim1", [64, 128, 256])
    hidden_dim2 = trial.suggest_categorical("hidden_dim2", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    activation_fn = trial.suggest_categorical("activation_fn", ["relu", "tanh"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = TabularNN(
        input_dim=100,
        hidden_dims=[hidden_dim1, hidden_dim2],
        dropout_rate=dropout_rate,
        activation_fn=activation_fn,
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Train for 10 epochs
    for epoch in range(10):
        train_one_epoch(model, optimizer, criterion, train_loader, device)
        val_loss = evaluate(model, val_loader, device)  # now returns MSE loss

        trial.report(val_loss, epoch)  # minimize val_loss

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_loss


# Launch the Optimization

In [25]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)


[I 2025-07-08 19:16:09,987] A new study created in memory with name: no-name-8df387bc-cc8d-4705-86eb-621ba5a28143
[I 2025-07-08 19:16:11,805] Trial 0 finished with value: 0.49564458915966597 and parameters: {'hidden_dim1': 256, 'hidden_dim2': 64, 'dropout_rate': 0.305106242852624, 'lr': 0.0006792903527009997, 'batch_size': 64, 'activation_fn': 'tanh'}. Best is trial 0 with value: 0.49564458915966597.
[I 2025-07-08 19:16:13,579] Trial 1 finished with value: 0.5952009236987456 and parameters: {'hidden_dim1': 64, 'hidden_dim2': 64, 'dropout_rate': 0.15162067908238042, 'lr': 0.00014813431803033553, 'batch_size': 64, 'activation_fn': 'tanh'}. Best is trial 0 with value: 0.49564458915966597.
[I 2025-07-08 19:16:14,686] Trial 2 finished with value: 0.4056628770640117 and parameters: {'hidden_dim1': 256, 'hidden_dim2': 128, 'dropout_rate': 0.04163344505749983, 'lr': 0.0004348654244197602, 'batch_size': 128, 'activation_fn': 'tanh'}. Best is trial 2 with value: 0.4056628770640117.
[I 2025-07-08

#  Print and Save Best Result

In [29]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [30]:
print("Best hyperparameters:", study.best_params)
print(f"Best validation accuracy: {1.0 - study.best_value:.4f}")
best_params = study.best_params
best_model = TabularNN(
    input_dim=100,
    hidden_dims=[best_params["hidden_dim1"], best_params["hidden_dim2"]],
    dropout_rate=best_params["dropout_rate"],
    activation_fn=best_params["activation_fn"],
).to(device)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)

optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()
# Retrain
for epoch in range(10):
    train_one_epoch(best_model, optimizer, criterion, train_loader, device)

torch.save(best_model.state_dict(), "best_tabular_model.pt")


Best hyperparameters: {'hidden_dim1': 256, 'hidden_dim2': 128, 'dropout_rate': 0.009378706712847181, 'lr': 0.006106405400676002, 'batch_size': 64, 'activation_fn': 'tanh'}
Best validation accuracy: 0.7363
